Imports and constants

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from PIL import Image
from pathlib import Path
from tqdm.auto import tqdm

import torch
import torch.utils.data as thd
import torch.nn as nn

from sklearn.metrics import fbeta_score, precision_score, recall_score
from scipy.ndimage.filters import gaussian_filter1d

import matplotlib.pyplot as plt
from collections import defaultdict
import os
import gc
from operator import itemgetter
from pympler import tracker

# Ignore SKLearn warnings
import warnings
from sklearn.exceptions import UndefinedMetricWarning
warnings.filterwarnings(action='ignore', category=UndefinedMetricWarning)

BATCH_SIZE = 256
BUFFER = 10  # Buffer size in both dimensions: x and y. Effective patch size is [BUFFER * 2 + 1, BUFFER * 2 + 1, Z_DIM].
SLICES = 65
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
LEARNING_RATE = 0.3
Z_START = 24
Z_END = 34
Z_STEP = 1
Z_DIM = (Z_END - Z_START) // Z_STEP
TRAIN_ON_FRAGMENTS = [1, 2]
VAL_FRAGMENT = 3
DISABLE_TQDM = False

MAX_TRAIN_STEPS = 300000000
MAX_VAL_STEPS = 1000
PRINT_EVERY = 20000000

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


First, let's see if we can fit all one full fragment into memory at once.

In [26]:
def pad_array(array):
    padding = (
        (BUFFER, BUFFER),
        (BUFFER, BUFFER),
    )
    return np.pad(array, padding)

def load_fragment_surface(fragment, split='train'):
    print("Loading fragment %s surface" % fragment)
    surface_path = Path("/kaggle/input/vesuvius-challenge-ink-detection/%s/%s/surface_volume" % (split, fragment))
    return np.array([
        (pad_array(np.array(Image.open(f))) / (2 ** 16)).astype('float16')
        for f in tqdm(sorted(surface_path.rglob("*.tif"))[Z_START:Z_END:Z_STEP], disable=DISABLE_TQDM)
    ])

def load_mask(fragment, split='train'):
    print("Loading fragment %s mask" % fragment)
    mask_path = Path("/kaggle/input/vesuvius-challenge-ink-detection/%s/%s/mask.png" % (split, fragment))
    return pad_array(np.array(Image.open(mask_path)))

def load_inklabels(fragment, split='train'):
    print("Loading fragment %s labels" % fragment)
    inklabels_path = Path("/kaggle/input/vesuvius-challenge-ink-detection/%s/%s/inklabels.png" % (split, fragment))
    return pad_array(np.array(Image.open(inklabels_path)))

In [39]:
class SingleFragmentDataset(thd.Dataset):
    def __init__(self, fragment, split_name='train'):
        assert split_name in ['train', 'val', 'test']
        self.split_name = split_name
        directory = 'test' if split_name == 'test' else 'train'
        self.surface = load_fragment_surface(fragment, directory)
        print(self.surface.dtype)
        self.mask = load_mask(fragment, directory)
        self.inklabels = load_inklabels(fragment, directory) if self.split_name != 'test' else None
        self.pixels = np.stack(np.where(self.mask == 1), axis=1)
    
    def __len__(self):
        return self.pixels.shape[0]
    
    def get_pixel_number(self, y, x):
        return 1 + (y - BUFFER) * (self.surface.shape[2] - 2 * BUFFER) + (x - BUFFER)
    
    def __getitem__(self, index):
        y, x = self.pixels[index]
        y_start = y - BUFFER
        y_end = y + BUFFER + 1
        x_start = x - BUFFER
        x_end = x + BUFFER + 1
        patch_surface = np.s_[:, y_start:y_end, x_start:x_end]
        patch_labels = np.s_[y_start:y_end, x_start:x_end]
        surface = self.surface[patch_surface].astype(np.float32)
        labels = self.inklabels[y, x].reshape((1, )).astype(np.float32) if self.split_name != 'test' else None
        pixel_number = self.get_pixel_number(y, x)
        return {
            'train': (surface, labels),
            'val': (surface, labels, pixel_number),
            'test': (surface, pixel_number)
        }[self.split_name]

Load train dataset - fragment 1

In [40]:
train_dataset = SingleFragmentDataset(3, split_name='val')

Loading fragment 3 surface


  0%|          | 0/10 [00:00<?, ?it/s]

float16
Loading fragment 3 mask
Loading fragment 3 labels


List pixels with ink

In [41]:
pixels_with_ink = np.where(train_dataset.inklabels == 1.)

pixel_numbers = []

for p in tqdm(list(zip(*pixels_with_ink)), disable=DISABLE_TQDM):
    pixel_numbers.append(train_dataset.get_pixel_number(*p))

  0%|          | 0/3172466 [00:00<?, ?it/s]

Define serializing logic

In [42]:
# [0, 1, 2, 4, 5]
# [-INF, 0, 1, 2, 4, 5, INF]
# [(0, -INF), (1, 0), (2, 1), (4, 2), (5, 4), (INF, 5)]
# starts: [0, 4, INF]
# ends: [-INF, 2, 5]

def rle(sorted_pixels):
    print("Encoding RLE")
    INF = 10**15
    sorted_pixels = [-INF] + sorted_pixels + [INF]
    zipped = list(zip(sorted_pixels[1:], sorted_pixels[:-1]))
    starts = [p[0] for p in zipped if p[0] != p[1] + 1]
    ends = [p[1] for p in zipped if p[0] != p[1] + 1]
    pairs = [(p[0], p[1] - p[0] + 1) for p in zip(starts[:-1], ends[1:])]
    return ' '.join(list(map(lambda p: "%d %d" % (p[0], p[1]), pairs)))

print(rle([0, 1, 2, 4, 5]))
print(rle([]))
print(rle([1, 3, 5, 7]))
# assert rle([0, 1, 2, 4, 5]) == "0 3 4 2"

Encoding RLE
0 3 4 2
Encoding RLE

Encoding RLE
1 1 3 1 5 1 7 1


Create a CSV

In [43]:
pixel_numbers.sort()

submission = defaultdict(list)
submission["Id"].append('3')
submission["Predicted"].append(rle(pixel_numbers))
    
pd.DataFrame.from_dict(submission).to_csv("/kaggle/working/submission.csv", index=False)

Encoding RLE


In [44]:
pd.DataFrame.from_dict(submission)

,Id,Predicted
0,3,4323222 18 4328469 21 4333717 24 4333757 7 433...


In [46]:
!diff -bsq /kaggle/working/submission.csv /kaggle/input/vesuvius-challenge-ink-detection/train/3/inklabels_rle.csv

Files /kaggle/working/submission.csv and /kaggle/input/vesuvius-challenge-ink-detection/train/3/inklabels_rle.csv are identical
